# Logging进阶篇  

和基础篇比，本篇才算是打开了logging的大门，基础篇的很多操作其实内部都做了不少默认动作，尤其是basicConfig，所做的内容其实很多  
本篇从基础介绍了logging 模块的基本类，通过对基础类的实例化来创建日志记录器，虽然相对基础篇更复杂，但其实更加符合python的思维。

logging 底层有四个大类：
* Logger: 也就是日志记录器类，实例化后直接用来操作各种日志记录操作
* Handler: 用于确定将日志记录输出到哪里，一个日志记录器可以有多个handler
* Filter: 更细粒度的区分对什么日志进行输出
* Formatter: 更细粒度的确定日志输出格式

最终日志记录器生成一个 LogRecord 实例，从而完成日志输出

## 整个logging的流程

![LoggingFlow](https://docs.python.org/3/_images/logging_flow.png)

流程图可以表述成以下内容：
* logging call (请求日志记录)
* 创建 LogRecord 实例
* 判断日志记录器的 filter 是否拒绝了该记录？
* 将日志记录传输给该日志记录器的所有handlers
    * handlers 判断 LogRecord 是否符合自己的level
    * handlers 的 filters 是否命中了该条 LogRecord
    * 输出
* 判断该日志记录器，是否有传输（广播）的性质
* 如果有，判断logger 是否有父logger，有的话将LogRecord继续传给父logger的handlers

所谓父logger，在Logger会讨论

## Logger  

* 实例化对象，就是logger，日志记录器
* 每个日志记录器都需要有个名字
    * 命名规则是可以有继承关系的，比如名字为 scan 的日志记录器，就是名字为 scan.pdf, scan.text 的父logger，当调用scan.pdf记录日志的时候，LogRecord同样会被传入给scan的handlers，被判断要不要被输出在一些地方（这个功能太高级了，暂时用不到）
    * 继承规则有一个用处是确定该logger的默认等级，当这个logger的等级没有被明确是什么的时候，会尝试去寻找父节点的level是什么，一直追查到最终根节点的logger，根节点肯定会有明确的level
* 名字原则上没有任何要求，仅仅是用来代表你知道这个日志出自哪里
* 但是最常见的命名logger的方式还是： logger = logging.getLogger(\_\_name\_\_) 因为这样可以明确的知道这条日志来自哪里
* 既然日志记录器的命名是可以继承的，那么就存在一个根节点的名字，根节点的名字就叫做 'root'，如果直接用logging来call，默认就是用这个logger
* logger 用logging.getLogger(name) 的形式获得，如果这个 name 在整个进程（全局）中已经被创建过，则会返回原有的logger，否则会创建一个新的logger，但是 'root' 这个字符除外，他不会调用根节点的logger， 仍然会创建一个新的，名字也叫 root 的子节点logger，因此实际中不建议创建名字为 root 的日志记录器


In [1]:
import logging
# 直接logging的日志记录器，默认的形式
logging.warning('This is a default warning')

In [1]:
# 需要重启解释器
import logging 
# 名字为__name__的日志记录器，默认的形式'
logger = logging.getLogger(__name__)
logger.warning('This is a __name__ warning')

# 名字为 root 的日志记录器，格式与上面的一致
logger_r = logging.getLogger('root')
logger_r.warning('This is a warning by a logger named root')

# logging 默认的根节点日志
logging.warning('This is a default warning')

This is a __name__ warning
This is a warning by a logger named root


可以看到 logger_r 和 根节点日志记录器，并不是同一个日志记录器

### Logger的工作内容，有个三个层面
* 日志各种 call
* 确定一个基本的输出 level：类似 warning，info 等 （其实就是一个基本的filter）
* 将满足level的log传输给相应的handler

这里可以看到，除了不能绑定 format，logger 可以与 filter和hanler 绑定完成工作

### 最常用的logger的函数
* Logger.setLevel(): 确定输出level
* Logger.addHandler() / Logger.removeHandler(): 添加 Handler
* Logger.addFilter() / Logger.removeFilter(): 添加Filter

当然你不需要每次创建新的日志记录器的时候都调用这些方法：
* 在python 3.2 之后，如果没有指定过handler，那么会使用 logging.lastResort，但是这不代表 logging.lastResort 被绑定在该logger上了
* 如果没有 setLevel 或者 指定过 filter ，那么logger本身的筛选就是none，对所有的信息都可以通过，真正起作用的是各个 handler 中的 filter，也就是level

In [2]:
logging.lastResort

<_StderrHandler stderr (WARNING)>

### 当Logger被配置完成以后
* Logger.debug()/Logger.info()/Logger.warning()/Logger.error()/Logger.critical()
    * 创建相应的 LogRecord
    * 有一个参数： exc_info 是用来判定log是否要输出 exception 信息的
* Logger.expection()
    * 和 logger.error() 相似，区别就在于exception 同时会输出 stack trace
    * 只有 exception handler 才可以用
* Logger.log()
    * 可以设定 loglevel，这个loglevel应该是可以用数字的，适用于整个日志等级不仅仅是简单的五个level，需要更多等级时候的个性化配置

In [1]:
import logging
# 验证 logging.warning logging.error logging.exception 的关系
try:
    int('s')
except:
    logging.exception('error:')

try:
    int('s')
except:
    logging.error('error but no exception')

try:
    int('s')
except:
    logging.error('error with exception:', exc_info=True)

try:
    int('s')
except:
    logging.error('error but no exception', exc_info=False)

try:
    int('s')
except:
    logging.warning('warning with exception', exc_info=True)

ERROR:root:error:
Traceback (most recent call last):
  File "<ipython-input-1-5ce02062a8d4>", line 4, in <module>
    int('s')
ValueError: invalid literal for int() with base 10: 's'
ERROR:root:error but no exception
ERROR:root:error with exception:
Traceback (most recent call last):
  File "<ipython-input-1-5ce02062a8d4>", line 14, in <module>
    int('s')
ValueError: invalid literal for int() with base 10: 's'
ERROR:root:error but no exception
Traceback (most recent call last):
  File "<ipython-input-1-5ce02062a8d4>", line 24, in <module>
    int('s')
ValueError: invalid literal for int() with base 10: 's'


### 关于传播
* 子日志记录器会把LogRecord传递给父日志记录器，所以没有必要将所有的handlers赋给所有的logger，合理利用这种层级关系，可以更有效
* 如果要修改这种传播性质，可以用 Logger.propagate
* 这里要注意的是，logger如果没有addHandler过，那么会调用logging.lastResort 来做输出，这并不代表handler被绑定在logger上了

In [1]:
# 需要重启解释器
# 新日志记录器的父记录器是 root，并且有传播的性质
import logging

# 创建一个日志记录器
logger = logging.getLogger('foo')
# 会打印一条信息，虽然有传播性质，但是因为 root 没有初始化，也就是没有绑定过handler，所以不会输出
logger.warning('warning from foo')

warning from foo


In [2]:
# 给根记录器也绑定一个handler
logging.basicConfig(format = '%(name)s: %(message)s',level = 'INFO')
logging.info('init root')

root: init root


In [3]:
# 会发送一条，且仅来自root，因为 logging.lastResort 并没有被使用
logger.warning('warning from foo')

foo: warning from foo


In [4]:
# 给 logger 也绑定handler，方便起见就使用 logging.lastResort
logger.addHandler(logging.lastResort)
# 这里会发送两条，一条来自 foo， 一条来自 root
logger.warning('warning from foo')
# 另外我们可以看到因为本身这个 LogRecord 是有 foo 创建的，所以虽然是root发出的信息，还是会显示 name = foo

warning from foo
foo: warning from foo


In [5]:
# 这里会出现一条，是来自 root 的，因为 foo 自己并没有设置 filter，info信息是可以通过的，之所以没输出是 foo对应的handler限制了输出
logger.info('info from foo')

foo: info from foo


In [6]:
# 对 foo setLevel
logger.setLevel('WARNING')
# 这里就不会出现任何记录，因为logger在最开始就过滤掉了本条记录，没有传给root
logger.info('info from foo')

## Handler

* 一个日志记录器可以把 LogRecord 传输给不同的输出内容，包括
    * 文件
    * HTTP GET/POST 
    * 邮件
    * sockets
    * queues (可能是进程之间的管道？)
    * 系统特定日志记录方法
    * ……任何其他自定义的handler
* 默认状态下没有handle会直接被赋予一个logger，如果使用basicConfig，那么会有一个传输给 console 或者 文件 的handle赋予root
* 如果没有使用basicConfig 直接使用了各种call 命令，则会创建一个到 console 的handler(当然也会创建一个默认的format)
* 正如流程图所示，每个handler自己是可以有filter 的，也就是说你可以给一个 LogRecord 设置多个 handler，每个handler有不同的筛选条件，比如将所有的日志信息输入文件，但是仅在 console 输出warning 以上的信息

### Handle 标准操作

其他的操作，因不同的Handler 会有不同
* setLevel(): 设置level
* setFormatter(): 设置 format
* addFilter() / removeFilter(): 设置特殊的Filter

### 标准库 Handler 列举

各个Handler的使用方法参考各种接口

* StreamHandler: 传递数据给console，默认是 sys.stderr， 可以指定其他的流，可以直接在basicConfig中指定
* FileHandler: 可以设定文件，写入的方式（a，w），编码，延迟（直到有第一个call才创建文件），可以直接在basicConfig中指定
* BaseRotatingHandler: 循环文件的Base类，不太需要直接使用
    * RotatingFileHandler: 可以循环写入文件，按照文件的大小分割（默认文件大小可以无限大），每次被写的文件都是 .log ，当rollover时，会被命名成 .log.1 其他的名字会被重命名到 .2  .3 ...
    * TimedRotatingFileHandler: 可以循环写入文件，按照时间分割，可以按照 各种 时间间隔 来间隔，也可以用 'midnight' , 可以设置文件后缀,具体如果需要了解还是官方文档的好。
* SocketHandler
* ……

## formatters

## filters